In [ ]:
# python -m venv policyengine_venv
# source policyengine_venv/bin/activate
# make sure to select kernel to be in the above environment
# Install ipykernel if not already installed
!pip install ipykernel
!pip install python-dotenv
# Register the environment as a kernel
!python -m ipykernel install --user --name=policyengine_venv --display-name="Python (PolicyEngine)"
# import sys
# print(sys.executable)  # Should show a path inside your policyengine_venv directory
#Install policy engine
!pip install policyengine-uk

In [ ]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the tokens
token = os.environ.get("HUGGING_FACE_TOKEN")

# Now you can use the token
print(f"Token loaded: {token[:4]}...{token[-4:]}")  # Prints first and last 4 chars for verification

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
from policyengine_core.parameters import Parameter

import pandas as pd
# LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
# sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
#sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
 

In [2]:
def avg_gain_gainer(baseline, reformed, eval_year):
    net_income_baseline = baseline.calculate("household_net_income", eval_year)
    net_income_reformed = reformed.calculate("household_net_income", eval_year)
    diff = net_income_reformed*net_income_reformed.weights - net_income_baseline*net_income_baseline.weights
    gainers = (diff > 0).sum()
    total=net_income_reformed.sum()-net_income_baseline.sum()
    print(f"Avg gain of gainers {eval_year}: {total/gainers}")

In [3]:
def povertycalc(baseline, eval_year, reformed):
    # Step 1: Calculate in_poverty for everyone
    baseline_pov = baseline.calculate("in_relative_poverty_ahc", eval_year, map_to="person", use_weights=True)
    ages = baseline.calculate("age", eval_year)
    weights = baseline.get_weights("age", eval_year)
    # Step 2: Create mask for children (age < 16 or < 18 as appropriate)
    is_child = ages < 16  # or < 18 depending on definition
    # Step 3: Get child-level poverty status and weights
    child_poverty = baseline_pov[is_child]
    child_weights = weights[is_child]
    # Step 4: Calculate total number and percent of children in poverty
    num_children = child_weights.sum()
    num_children_in_poverty = (child_poverty).sum()
    percent_in_poverty = num_children_in_poverty / num_children
    # print(f"Number of children: {num_children:,.0f} in {eval_year}, baseline")
    # print(f"Number of children in poverty: {num_children_in_poverty:,.0f}in {eval_year}, baseline")
    # print(f"Percent of children in poverty: {percent_in_poverty:.2%}in {eval_year}, baseline")
    
    # Step 1: Calculate in_poverty for everyone
    baseline_pov = reformed.calculate("in_relative_poverty_ahc", eval_year, map_to="person", use_weights=True)
    ages = reformed.calculate("age", eval_year)
    weights = reformed.get_weights("age", eval_year)
    # Step 2: Create mask for children (age < 16 or < 18 as appropriate)
    is_child = ages < 16  # or < 18 depending on definition
    # Step 3: Get child-level poverty status and weights
    child_poverty = baseline_pov[is_child]
    child_weights = weights[is_child]
    # Step 4: Calculate total number and percent of children in poverty
    num_children = child_weights.sum()
    num_children_in_poverty_after = (child_poverty).sum()
    percent_in_poverty_after = num_children_in_poverty_after / num_children
    # print(f"Number of children: {num_children:,.0f} in {eval_year}, reformed")
    # print(f"Number of children in poverty: {num_children_in_poverty_after:,.0f} in {eval_year}, reformed")
    # print(f"Percent of children in poverty: {percent_in_poverty_after:.2%} in {eval_year}, reformed")
    print(f"Reduction in child poverty: {num_children_in_poverty - num_children_in_poverty_after:,.0f} in {eval_year}, reformed")
    

In [ ]:
# Level 1: 20 GBP pw uplift to child element of UC
start_date= "2029-01-01"
eval_year=2029

def change_tax_parameters(parameters):
    add=20.00*52/12
    parameters.gov.dwp.universal_credit.elements.child.amount.update(
        start=instant(start_date), value=orig_param+add
        )
    parameters.gov.dwp.universal_credit.elements.child.first.higher_amount.update(
        start=instant(start_date), value=orig_param_higher+add
    )
    return parameters


class reform(Reform):
    def apply(self):
        #self.modify_parameters(change_tax_parameters)
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
Microsimulation()
orig_param = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.elements.child.amount(start_date)
orig_param_higher = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.elements.child.first.higher_amount(start_date)
print(orig_param,orig_param_higher )
reformed = Microsimulation(reform=reform)

revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in {eval_year}")
povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)


In [ ]:
#Level1b:  Work allowance increased to 1000 GBP  pa
start_date= "2029-01-01"
eval_year=2029

def change_tax_parameters(parameters):
    add=1000.00/12  # Monthly increase

    parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing.update(
        start=instant(start_date), value=orig_without_housing+add
    )
    parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing.update(
        start=instant(start_date), value=orig_with_housing+add
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
orig_without_housing = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing(start_date)
orig_with_housing = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing(start_date)
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calc("gov_spending", eval_year).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in {eval_year}")

povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)



In [ ]:
#Level 1c: Change child limit
start_date= "2026-01-01"
eval_year=2026
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.limit.child_count.update(
        start=instant(start_date), value=inf
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)


In [ ]:
#Level 2b UNDER 5s premium
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_bonus_for_under5 import reform
eval_year=2029
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
print(reformed.calculate("num_households_with_under5", eval_year).sum())
print(reformed.calculate("universal_credit", eval_year).mean())
print(baseline.calculate("universal_credit", eval_year).mean())
print(f"Number of households with at least one child under 5: {under5_households}")
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)

In [4]:
#Level 3 ADD 30 HOUR WORK BONUS
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_30hour_bonus_v2 import reform
eval_year=2026
baseline = Microsimulation()
NMW = baseline.tax_benefit_system.parameters.gov.hmrc.minimum_wage.OVER_24(eval_year)
print(NMW)
reformed = Microsimulation(reform=reform)
# Debug: Check if UC is actually changing
baseline_uc = baseline.calculate("universal_credit", eval_year)
reformed_uc = reformed.calculate("universal_credit", eval_year)
print(reformed.calculate("universal_credit", eval_year).mean())
print(baseline.calculate("universal_credit", eval_year).mean())
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)

12.21
1069.0
1926.7440341598885
1911.9556332887444
Cost: £0.6bn
Reduction in child poverty: 46,513 in 2026, reformed
Avg gain of gainers 2026: 1069.0058541234218


In [ ]:
from policyengine_uk.system import system
parameters = system.parameters
print(parameters.gov.hmrc.minimum_wage.OVER_24)

**OLD STUFF**

In [ ]:
def calc_childpov(baseline,year):
    # Step 1: Calculate in_poverty for everyone
    baseline_pov = baseline.calculate("in_relative_poverty_ahc", year, map_to="person", use_weights=True)
    ages = baseline.calculate("age", year)
    weights = baseline.get_weights("age", year)
    # Step 2: Create mask for children (age < 16 or < 18 as appropriate)
    is_child = ages < 16  # or < 18 depending on definition
    print(ages.min())
    # Step 3: Get child-level poverty status and weights
    child_poverty = baseline_pov[is_child]
    child_weights = weights[is_child]
    # Step 4: Calculate total number and percent of children in poverty
    num_children = child_weights.sum()
    num_children_in_poverty = (child_poverty).sum()
    percent_in_poverty = num_children_in_poverty / num_children
    print(f"Number of children: {num_children:,.0f}")
    print(f"Number of children in poverty: {num_children_in_poverty:,.0f}")
    print(f"Percent of children in poverty: {percent_in_poverty:.2%}")
    return num_children_in_poverty

In [ ]:


net_income_baseline = baseline.calculate("household_net_income", 2026)
net_income_reformed = reformed.calculate("household_net_income", 2026)
net_income_baseline_29 = baseline.calculate("household_net_income", 2029)
net_income_reformed_29 = reformed.calculate("household_net_income", 2029)
total_net_income_baseline = baseline.calculate("household_net_income", 2029).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2029).sum()
diff=total_net_income_reformed-total_net_income_baseline
print(f"Total net income in 2029: {diff/1e9:.2f}bn")


In [ ]:
diff = net_income_reformed*net_income_reformed.weights - net_income_baseline*net_income_baseline.weights
gainers = (diff > 0).sum()
total=net_income_reformed.sum()-net_income_baseline.sum()
print(f"Avg gain of gainers 2026: {total/gainers}")

##RUN 2 CHILD LIMIT REFORM: 

In [ ]:
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "age", "household_net_income", "household_income_decile", "in_poverty", "household_tax", "household_benefits"]
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
reformed_person_df = reformed.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
difference_person_df = reformed_person_df - baseline_person_df
total_net_income_baseline = baseline.calculate("household_net_income", 2026).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2026).sum()

net_cost = total_net_income_reformed - total_net_income_baseline

print(f"This reform would cost £{net_cost / 1e9:.1f}bn")

In [ ]:

def modify_parameters(parameters):
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_OLD.update(start=instant("2024-01-01"), value=738.82)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_YOUNG.update(start=instant("2024-01-01"), value=618.51)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_OLD.update(start=instant("2024-01-01"), value=528.74)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_YOUNG.update(start=instant("2024-01-01"), value=452.11)
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)

baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2025).sum() - baseline.calc("gov_spending", 2025).sum()
f"Revenue: £{round(revenue / 1e+9, 1)}bn"


In [ ]:
# Step 1: Make sure to include the weights column in your variables
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "household_weight", "person_weight"]  # Include the weight variable
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, period=2032, use_weights=True).astype(float)
weighted_persons= baseline_person_df["person_weight"].sum()

# Step 2: Create a household-level dataframe by dropping duplicate household IDs
household_df = baseline_person_df.drop_duplicates('household_id')

# Step 3: Calculate the weighted number of households
# Sum the weights column, not the entire dataframe
weighted_households = household_df["household_weight"].sum()

print(f"Total weighted number of households: {weighted_households:,.0f}")
print(f"Total weighted number of people: {weighted_persons:,.0f}")

In [ ]:
# Level 1: 20 GBP pw uplift to child element of UC

def change_tax_parameters(parameters):
    latest_value_amount = parameters.gov.dwp.universal_credit.elements.child.amount("2026-01-01")
    print(latest_value_amount)
    parameters.gov.dwp.universal_credit.elements.child.amount.update(
        start=instant("2026-01-01"), value=292.81+80.00
    )
    parameters.gov.dwp.universal_credit.elements.child.first.higher_amount.update(
        start=instant("2026-01-01"), value=339.00+80.00
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)


revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calculate("gov_spending", 2026).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in 2026")


In [ ]:
from IPython.display import Markdown
from policyengine_uk.system import system
from policyengine_core.parameters import Parameter
import pandas as pd

parameters = system.parameters

markdown = """# UK parameters

This page shows a list of available parameters for reforms for the UK model.

We exclude from this list:

* Some parameters without documentation (we're a large, fast-growing model maintained by a small team- we're working on it!)
* Abolition parameters, which mirror each household property and allow the user to set the value of the property to zero (these take the format `gov.abolitions.variable_name`) because these roughly triple the size of this list and are repetitive.
"""
i = 0
for parameter in parameters.get_descendants():
    if isinstance(parameter, Parameter):
        if not parameter.metadata.get("label"):
            continue
        if ".abolitions." in parameter.name:
            continue
        if type(parameter(2025)) not in (int, float, bool):
            continue
        markdown += f"### {parameter.name}\n"
        markdown += f"**Label:** {parameter.metadata.get('label')}\n\n"
        if parameter.description:
            markdown += f"{parameter.description}\n\n"
        markdown += f"**Type:** {type(parameter(2025)).__name__}\n\n"
        markdown += f"**Current value:** {parameter(2025)}\n\n---"
        markdown += "\n\n"
        print(markdown) 
        i += 1

with open("./parameters_uk.md", "w+") as f:
    f.write(markdown)
   

In [ ]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
year=2026
# Step 1: Calculate in_poverty for everyone
baseline_pov = baseline.calculate("in_relative_poverty_ahc", year, map_to="person", use_weights=True)
ages = baseline.calculate("age", year)
weights = baseline.get_weights("age", year)
# Step 2: Create mask for children (age < 16 or < 18 as appropriate)
is_child = ages < 16  # or < 18 depending on definition
print(ages.min())
# Step 3: Get child-level poverty status and weights
child_poverty = baseline_pov[is_child]
child_weights = weights[is_child]
# Step 4: Calculate total number and percent of children in poverty
num_children = child_weights.sum()
num_children_in_poverty = (child_poverty).sum()
percent_in_poverty = num_children_in_poverty / num_children
print(f"Number of children: {num_children:,.0f}")
print(f"Number of children in poverty: {num_children_in_poverty:,.0f}")
print(f"Percent of children in poverty: {percent_in_poverty:.2%}")
 
# Step 1: Calculate in_poverty for everyone
baseline_pov = reformed.calculate("in_relative_poverty_ahc", year, map_to="person", use_weights=True)
ages = reformed.calculate("age", year)
weights = reformed.get_weights("age", year)
# Step 2: Create mask for children (age < 16 or < 18 as appropriate)
is_child = ages < 16  # or < 18 depending on definition
print(ages.min())
# Step 3: Get child-level poverty status and weights
child_poverty = baseline_pov[is_child]
child_weights = weights[is_child]
# Step 4: Calculate total number and percent of children in poverty
num_children = child_weights.sum()
num_children_in_poverty_after = (child_poverty).sum()
percent_in_poverty_after = num_children_in_poverty / num_children
print(f"Number of children: {num_children:,.0f}")
print(f"Number of children in poverty: {num_children_in_poverty_after:,.0f}")
print(f"Percent of children in poverty: {percent_in_poverty_after:.2%}")
print(f"Reduction in child poverty: {num_children_in_poverty - num_children_in_poverty_after:,.0f}")

In [ ]:
from policyengine_uk.system import system

parameters = system.parameters